In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr  7 15:11:50 2020

@author: 12036
"""

from numpy import load
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

dict_data = load('/Users/apple/Desktop/graduate/Machine_Learning_Theory/MRI/npz/OAS1_0019_MR.npz')
data = dict_data['arr_0']

dict_data1 = load('/Users/apple/Desktop/graduate/Machine_Learning_Theory/MRI/npz/OAS1_0019_MRseg.npz')
data2 = dict_data1['arr_0']


dict2_data =  load('/Users/apple/Desktop/graduate/Machine_Learning_Theory/MRI/npz/OAS1_0019_MR_skullseg.npz')
data3 = dict2_data['arr_0']

dict3_data = load('/Users/apple/Desktop/graduate/Machine_Learning_Theory/MRI/npz/npzOAS1_0019_MRobstruct_seg.npz')
data4 = dict3_data['arr_0']

segment_ven = []               
segment_skull = []                
segment_ob = []
segment_ven = np.matrix(np.nonzero(data2 > 0))
segment_skull = np.matrix(np.nonzero(data3 > 0))
segment_ven = segment_ven.T
segment_skull = segment_skull.T
segment_ob = np.matrix(np.nonzero(data4 > 0))
segment_ob = segment_ob.T






<Figure size 640x480 with 1 Axes>

[[  0  14  90]
 [  0  14  91]
 [  0  14  92]
 ...
 [173 110  89]
 [173 110  90]
 [173 110  91]]


<Figure size 640x480 with 1 Axes>

In [ ]:
from needle_path_updated import rand_pt, skull_point, vent_point, rand_needle_path

sp = skull_point(segment_skull)
vp = vent_point(sp, segment_ven)
line_arr = rand_needle_path(segment_skull,segment_ven)
ob_filtered, path_len = find_obs(segment_ob,line_arr,sp,vp)
sum_d, d_mean = ob_distance(ob_filtered,line_arr)


[[147 152  58]]
[[ 65  19 124]]


In [ ]:
evaluate_function(path_len,d_mean)

7.89068913338199

In [ ]:
def find_obs(segment_ob,line_arr,skull_point,ven_point):
    """This function filters the obstacle points between the ventricle and skull"""
    
    xmin = min(sp[0,0],vp[0,0])
    xmax = max(sp[0,0],vp[0,0])
    xc = (xmin + xmax)/2
    ymin = min(sp[0,1],vp[0,1])
    ymax = max(sp[0,1],vp[0,1])
    yc = (ymin + ymax)/2
    zmin = min(sp[0,2],vp[0,2])
    zmax = max(sp[0,2],vp[0,2])
    zc = (zmin + zmax)/2
    path_len = np.sqrt(np.sum(np.square(sp-vp)))
    radius = np.sqrt(np.sum(np.square(sp-vp))) /2 
    x_range = range(int(xc - radius), int(xc + radius + 1) )
    y_range = range(int(yc - radius), int(yc + radius + 1) )
    z_range = range(int(zc - radius), int(zc + radius + 1) )


    ob_filtered = [pt for pt in segment_ob if (pt[0,0] in x_range) 
                   & (pt[0,1] in y_range) &(pt[0,2] in z_range)]
    
                                             
    return ob_filtered, path_len  
    
    
############################################################################################################


def ob_distance(obst,line_arr):
    """This function calculates the random distance and its average value """
    
    import random
    ob_points = random.sample(list(obst),1000)
    sum_d = []
    
    for x0 in ob_points:
        
        x1 = line_arr[0]
        x2 = line_arr[-1]
        
        num_cross = np.cross((x2-x1), (x1-x0))
        num = np.linalg.norm(num_cross)
        
        denom = np.linalg.norm(x2-x1)
        d = num/denom
        sum_d.append(d)
        d_mean = np.mean(sum_d)
        
    
    return sum_d, d_mean

############################################################################################################

def evaluate_function(path_len,d_mean):
    
    score = d_mean-0.01*(path_len)**2
    
    return score


